## This notebook will help you train a latent Point-Cloud GAN.

(Assumes
latent_3d_points is in the PYTHONPATH and that a trained AE model exists)

In [1]:
# -*- coding: utf-8 -*-
import numpy as np
import os
import os.path as osp
import sys
import matplotlib.pylab as plt

BASE = os.path.dirname(os.path.abspath(os.path.dirname("file"))) # latent_3D
sys.path.append(BASE) 

from src.point_net_ae import PointNetAutoEncoder
from src.autoencoder import Configuration as Conf
from src.neural_net import MODEL_SAVER_ID

from src.in_out import snc_category_to_synth_id, create_dir, PointCloudDataSet, \
                                        load_all_point_clouds_under_folder

from src.general_utils import plot_3d_point_cloud
from src.tf_utils import reset_tf_graph

from src.vanilla_gan import Vanilla_GAN
from src.w_gan_gp import W_GAN_GP
from src.generators_discriminators import latent_code_discriminator_two_layers,\
latent_code_generator_two_layers

Instructions for updating:
Colocations handled automatically by placer.
Success


In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

Specify where the raw point-clouds and the pre-trained AE are.

In [13]:
# Top-dir of where point-clouds are stored.
class_name = 'chair'
ae_loss = 'chamfer'  
top_in_dir = 'data/shape_net_core_uniform_samples_2048/'    

# ae_configuration = '../data/single_class_ae/configuration'
# ae_configuration = osp.join(BASE, 'data', class_name, 'single_class_ae/configuration')
ae_configuration = 'single_class_ae_' + ae_loss
ae_configuration = osp.join(BASE, 'data', class_name, ae_configuration, 'configuration')

In [14]:
# Where to save GANs check-points etc.
top_out_dir = 'data/'
top_out_dir = osp.join(BASE, top_out_dir, class_name)
top_in_dir = osp.join(BASE, top_in_dir)

In [11]:
# Load point-clouds.
syn_id = snc_category_to_synth_id()[class_name]
class_dir = osp.join(top_in_dir , syn_id)
all_pc_data = load_all_point_clouds_under_folder(class_dir, n_threads=8, file_ending='.ply', verbose=True)
print ('Shape of DATA =', all_pc_data.point_clouds.shape)

6778 pclouds were loaded. They belong in 1 shape-classes.
Shape of DATA = (6778, 2048, 3)


In [12]:

# import tensorflow as tf
# import errno

# def make_sure_path_exists(path):
#     try:
#         os.makedirs(path)
#     except OSError as exception:
#         if exception.errno != errno.EEXIST:
#             raise
            

# dir_name = "GT_TXT"
# TXT_path = osp.join(top_out_dir, dir_name)

# make_sure_path_exists(TXT_path)

# L = all_pc_data.point_clouds[:]
# l = all_pc_data.num_examples

# print(l)

# for i in range(0,l):
#     xx = L[i][:, 0]
#     x = xx.tolist()
#     yy = L[i][:, 1]
#     y = yy.tolist()
#     zz = L[i][:, 2]
#     z = zz.tolist()

#     p = open(osp.join(TXT_path, 'GT'+str(i)+'.txt'), 'w+')
#     for k in range(0,2048):
        
#         p.write('%10f\t%10f\t%10f\n' %(x[k], y[k], z[k]))
#     p.close()

# print("Created GT.txt")


FileNotFoundError: [Errno 2] No such file or directory: 'data/chair/single_class_ae/configuration.pickle'

In [ ]:
L = all_pc_data.point_clouds[:]

import numpy as np
# np.save("/home/sohee/coding/latent_3d_points/result/chair/single_class_ae/ae_re.npy", reconstructions)
GT_name = class_name + '_GT.npy'
GT_path = osp.join(BASE, 'data', 'GT')
np.save(osp.join(GT_path, GT_name), L)